To-do for next week:

# FIX: Need to 2nd field with coalesce of client clean or client clean french (Use the right fields)

# Firm clean and firm clean fr work in a similar way; if they are both null, then they handle it in house

Next steps:

1. Use the right fields
2. Get the embeddings stored in Postgres (see how querying works)—if OK, great, else explore other storage mechanisms

^ for next week

3. Explore how this could be integrated into ETL and put on the site: <-- automate generation and storage
4. Integrate to the site as a beta feature: lead magnet, done > perfect
5. Examine other data sources (meetings; where they talk to one another)
   --> Bi-level filtering on both the semantic search of the "text body" and metadata through the existing databases


1. Load necessary libraries, read the CSV with the extracted table data, and load the environment files.


In [2]:
# Load libraries
import pkg_resources

pkg_resources.require("openai==0.28.1")
import pandas as pd
import openai
import os


# Read the CSV file
# input_file = "src/05_12_2024_grouping.csv"
# input_file = "src/05_12_2024_most_recent_regrs.xml"
input_file = "src/05_15_24_extract.xml"
new_file_name = "with_embeddings"

if input_file.endswith(".csv"):
    df = pd.read_csv(input_file)
    new_file_name = input_file.replace(".csv", "") + "_with_embeddings.csv"

elif input_file.endswith(".xml"):
    df = pd.read_xml(input_file)
    new_file_name = (
        input_file.replace(".xml", "").replace("src/", "output/")
        + "_with_embeddings.csv"
    )

# Load the environment files for .env
from dotenv import load_dotenv

load_dotenv()

# print(df)

True

2. Use OpenAI's new text-embedding-3-large model to generate an embedding based on the description field, storing it in a new column.


In [4]:
import openai
import os
import time
import random
import pandas as pd

# Set up OpenAI API credentials
openai.api_key = os.getenv("OPENAI_KEY")

# Print the current openai library version
print(openai.__version__)


def generate_embeddings(texts):
    embeddings = []
    responses = []
    for i in range(0, len(texts), 25):
        batch = texts[i : i + 25]
        response = openai.Embedding.create(input=batch, model="text-embedding-3-large")
        batch_embeddings = [item["embedding"] for item in response["data"]]
        embeddings.extend(batch_embeddings)
        responses.append(response)
        print(
            f"Processed {i+25 if i+25 < len(texts) else len(texts)} of {len(texts)} texts"
        )
        time.sleep(random.uniform(0.001, 0.5))  # Random wait between 1 to 500 ms
    return embeddings, responses


# Assuming 'df' is your DataFrame
# Modify texts so that it is a concatenation of the columns description, client_coalesced, firm_coalesced with labels
texts = df.apply(
    lambda row: f"description: {row['description']} \n\n client: {row['client_coalesced']} \n\n firm: {row['firm_coalesced']}",
    axis=1,
).tolist()
# FIX: Need to 2nd field with coalesce of client clean or client clean french
# We care more about the firm (lobbyist) in the data than the actual name of the people
# Firm clean and firm clean fr work in a similar way; if they are both null, then they handle it in house

# Registration is a contract between an organization (or itself), and an organization and an external lobby firm
# That contract generates meetings

embeddings, responses = generate_embeddings(texts)

# Update the DataFrame with the embeddings
df["embedding"] = embeddings

# Save the DataFrame to a new CSV file
df.to_csv(new_file_name, index=False)

print(f"Embeddings generated and saved to {new_file_name}")

0.28.1
Processed 25 of 2794 texts
Processed 50 of 2794 texts
Processed 75 of 2794 texts
Processed 100 of 2794 texts
Processed 125 of 2794 texts
Processed 150 of 2794 texts
Processed 175 of 2794 texts
Processed 200 of 2794 texts
Processed 225 of 2794 texts
Processed 250 of 2794 texts
Processed 275 of 2794 texts
Processed 300 of 2794 texts
Processed 325 of 2794 texts
Processed 350 of 2794 texts
Processed 375 of 2794 texts
Processed 400 of 2794 texts
Processed 425 of 2794 texts
Processed 450 of 2794 texts
Processed 475 of 2794 texts
Processed 500 of 2794 texts
Processed 525 of 2794 texts
Processed 550 of 2794 texts
Processed 575 of 2794 texts
Processed 600 of 2794 texts
Processed 625 of 2794 texts
Processed 650 of 2794 texts
Processed 675 of 2794 texts
Processed 700 of 2794 texts
Processed 725 of 2794 texts
Processed 750 of 2794 texts
Processed 775 of 2794 texts
Processed 800 of 2794 texts
Processed 825 of 2794 texts
Processed 850 of 2794 texts
Processed 875 of 2794 texts
Processed 900 of

3. Upsert embeddings into a Pinecone vector search database to query for results.


In [43]:
import pandas as pd
import time

# Define the embedding dimension and the path to your CSV file
EMBEDDING_DIMENSION = 3072

manuallyRepopulateDF = False
if manuallyRepopulateDF:
    # Read the dataframe
    df = pd.read_csv(new_file_name)
    # Promote first row to headers and drop it
    df.columns = df.iloc[0]
    df = df[1:]

from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone
pc = Pinecone(api_key="6c158534-e851-4152-8051-821eab050e00")

# Define your index name
index_name = "liq-registrations"

# # Connect to an existing index or create one if it doesn't exist
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws", region="us-east-1"  # Specify the cloud provider and region
        ),
    )  # Adjust metric as needed
index = pc.Index(name=index_name)

# Prepare the data for Pinecone

list_of_vectors = []
# Now, create a list in the format required by Pinecone for every 250 rows in the dataframe.
# Make that it its own list, and then append it to list_of_vectors.
increment = 0
for i in range(0, len(df), 100):
    vectors = [
        (
            str(row["matching_reg_id_enr"]).replace(",", ""),
            list(row["embedding"]),
            {
                # Following this pattern: description,matching_reg_id_enr,matching_reg_type_enr,matching_reg_num_enr,matching_en_client_org_corp_nm_an,matching_fr_client_org_corp_nm,matching_effective_date_vigueur,matching_end_date_fin,matching_rgstrnt_last_nm_dclrnt,matching_rgstrnt_1st_nm_prenom_dclrnt
                "description": row["description"],
                "matching_reg_id_enr": row["matching_reg_id_enr"].replace(",", ""),
                "matching_reg_type_enr": row["matching_reg_type_enr"],
                "matching_reg_num_enr": row["matching_reg_num_enr"],
                "matching_en_client_org_corp_nm_an": row[
                    "matching_en_client_org_corp_nm_an"
                ],
                "matching_fr_client_org_corp_nm": row["matching_fr_client_org_corp_nm"],
                "matching_effective_date_vigueur": row[
                    "matching_effective_date_vigueur"
                ],
                "matching_end_date_fin": row["matching_end_date_fin"],
                "matching_rgstrnt_last_nm_dclrnt": row[
                    "matching_rgstrnt_last_nm_dclrnt"
                ],
                "matching_rgstrnt_1st_nm_prenom_dclrnt": row[
                    "matching_rgstrnt_1st_nm_prenom_dclrnt"
                ],
                # "openai_response": row["openai_response"],
            },
        )
        for index, row in df[i : i + 100].iterrows()
    ]
    list_of_vectors.append(vectors)

# Upsert data into the Pinecone index
# Now, iterate through the list of vectors and pause to upsert into the Pinecone index. Make it count through each of the elements and print to console.
for i, vectors in enumerate(list_of_vectors):
    index.upsert(vectors)
    print(f"Upserted {i + 1} of {len(list_of_vectors)}")
    time.sleep(1)

Upserted 1 of 30
Upserted 2 of 30
Upserted 3 of 30
Upserted 4 of 30
Upserted 5 of 30
Upserted 6 of 30
Upserted 7 of 30
Upserted 8 of 30
Upserted 9 of 30
Upserted 10 of 30
Upserted 11 of 30
Upserted 12 of 30
Upserted 13 of 30
Upserted 14 of 30
Upserted 15 of 30
Upserted 16 of 30
Upserted 17 of 30
Upserted 18 of 30
Upserted 19 of 30
Upserted 20 of 30
Upserted 21 of 30
Upserted 22 of 30
Upserted 23 of 30
Upserted 24 of 30
Upserted 25 of 30
Upserted 26 of 30
Upserted 27 of 30
Upserted 28 of 30
Upserted 29 of 30
Upserted 30 of 30


#### ALTERNATE

3. Upsert embeddings into PostgreSQL.


In [9]:
# Contingency load from .CSV

# Load the data into df from the CSV file: 05_15_24_extract_with_embeddings.csv
df = pd.read_csv("output/05_15_24_extract_with_embeddings.csv")

# Drop the matching_rgstrnt_last_nm_dclrnt, matching_rgstrnt_1st_nm_prenom_dclrnt columns
df = df.drop(
    columns=["matching_rgstrnt_last_nm_dclrnt", "matching_rgstrnt_1st_nm_prenom_dclrnt"]
)

# Drop the  'matching_reg_type_enr', 'matching_reg_num_enr' columns
df = df.drop(columns=["matching_reg_type_enr", "matching_reg_num_enr"])

# Remove commas from the numbers presented in the matching_reg_id_enr column
df["matching_reg_id_enr"] = df["matching_reg_id_enr"].str.replace(",", "")

# Rename the columns, such that client_coalesced --> client_clean_coalesce and firm_coalesced --> firm_clean_coalesce
df = df.rename(
    columns={
        "client_coalesced": "client_clean_coalesce",
        "firm_coalesced": "firm_clean_coalesce",
    }
)

In [10]:
import os
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Database credentials
HOST = os.getenv("Host")
PORT = os.getenv("Port")
DATABASE = os.getenv("Database")
USERNAME = os.getenv("Username")
PASSWORD = os.getenv("Password")

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    host=HOST, port=PORT, dbname=DATABASE, user=USERNAME, password=PASSWORD
)

# Convert DataFrame to list of tuples
tuples = [(i,) + tuple(x) for i, x in enumerate(df.to_numpy())]

# Print the first row of the tuple, with a corresponding column name
print(tuples[0])

# Print the list of all column names in the DataFrame
print(df.columns)

# Define insert query
insert_query = """
INSERT INTO e_registration_linking (
    registration_linking_id,
    description,
    matching_reg_id_enr,
    client_clean,
    client_clean_fr,
    client_clean_coalesce,
    firm_clean,
    firm_clean_fr,
    firm_clean_coalesce,
    matching_effective_date_vigueur,
    matching_end_date_fin,
    embedding
) VALUES %s
"""

# Insert DataFrame rows into PostgreSQL table
with connection.cursor() as cursor:
    execute_values(cursor, insert_query, tuples)
    connection.commit()

# Close the connection
connection.close()

print("Data inserted successfully")

(0, '1. continue to create awareness and support for the indigenous screen office.', '{949975}', '{"Indigenous Screen Office"}', '{NULL}', '{"Indigenous Screen Office"}', '{"Public Affairs Counsel"}', '{NULL}', '{"Public Affairs Counsel"}', '{2024-04-15}', '{NULL}', '[0.009432810358703136, -0.0036308488342911005, -0.013157203793525696, -0.022415976971387863, -0.024434808641672134, 0.018604565411806107, 0.01862196810543537, 0.05238516628742218, -0.032092440873384476, 0.02311212569475174, 0.03668702021241188, -0.018726391717791557, 0.04082910344004631, 0.002105848863720894, -0.021110698580741882, -0.003284950042143464, -0.023686448112130165, 0.04082910344004631, 0.039854492992162704, -0.0434744656085968, 0.053185734897851944, -0.035520970821380615, -0.008427746593952179, 0.010920827277004719, 0.009485021233558655, -0.0005732346908189356, -0.006152211222797632, 0.021980885416269302, 0.0067569902166724205, 0.0012791726039722562, -0.008249358274042606, 0.03021284006536007, 0.024991726502776